In [28]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib
import os
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline

### Download all NetCDF files from a directory on Cal-Adapt Server

In [8]:
models = ['HadGEM2-ES']

In [23]:
ext = 'nc'
out_dir = 'data/zonal_stats'

In [10]:
def listFD(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

In [26]:
def download_nc(model, year, climvar):
    url = 'http://albers.cnr.berkeley.edu/data/scripps/loca/{}/rcp85/{}' .format(model, climvar)
    file_list = listFD(url, ext)
    for file in file_list:
        if str(year) in file:
            out_file = '{}/{}_{}_{}.nc' .format(out_dir, climvar, model, year)
            print('Downloading:', file, 'Saved as:', out_file)
            #urllib.urlretrieve (file, out_file)
            urllib.request.urlretrieve(file, out_file)

In [58]:
# Downloads 10 years of data from 2041-2050
for model in models:
    for year in np.arange(2041, 2050):
        download_nc(model, year, 'tasmax')

Downloading: http://albers.cnr.berkeley.edu/data/scripps/loca/HadGEM2-ES/rcp85/tasmin/tasmin_day_HadGEM2-ES_rcp85_r1i1p1_20410101-20411231.LOCA_2016-04-02.16th.CA_NV.nc Saved as: data/zonal_stats/tasmin_HadGEM2-ES_2041.nc
Downloading: http://albers.cnr.berkeley.edu/data/scripps/loca/HadGEM2-ES/rcp85/tasmin/tasmin_day_HadGEM2-ES_rcp85_r1i1p1_20420101-20421231.LOCA_2016-04-02.16th.CA_NV.nc Saved as: data/zonal_stats/tasmin_HadGEM2-ES_2042.nc


### Read all NetCDF files into an xarray


In [59]:
file_list = glob(os.path.join(out_dir, '*.nc'))
print(file_list)

['data/zonal_stats/tasmin_HadGEM2-ES_2041.nc', 'data/zonal_stats/tasmin_HadGEM2-ES_2042.nc', 'data/zonal_stats/tasmax_HadGEM2-ES_2041.nc', 'data/zonal_stats/tasmax_HadGEM2-ES_2042.nc']


In [30]:
def kelvin_to_f(value):
    return (value - 273.15) * 9 / 5 + 32

In [106]:
def create_dataframe():
    df_list = []
    for file_name in file_list:
        ds = xr.open_dataset(file_name)
        climvar = list(ds.data_vars)[0]
        arr = ds[climvar]
        data = arr.sel(lat=38.59375, lon=(360-121.46875), method='nearest')
        print(data.to_dataframe())
        ds.close()
    #return pd.DataFrame(averages, columns=[climvar, 'year', 'data'])

In [107]:
annual_df = create_dataframe()

                          lat        lon      tasmin
time                                                
2041-01-01 12:00:00  38.59375  238.53125  280.994629
2041-01-02 12:00:00  38.59375  238.53125  281.922668
2041-01-03 12:00:00  38.59375  238.53125  278.466675
2041-01-04 12:00:00  38.59375  238.53125  273.357147
2041-01-05 12:00:00  38.59375  238.53125  271.047394
2041-01-06 12:00:00  38.59375  238.53125  272.005310
2041-01-07 12:00:00  38.59375  238.53125  271.314758
2041-01-08 12:00:00  38.59375  238.53125  272.036987
2041-01-09 12:00:00  38.59375  238.53125  277.012146
2041-01-10 12:00:00  38.59375  238.53125  274.713135
2041-01-11 12:00:00  38.59375  238.53125  275.006622
2041-01-12 12:00:00  38.59375  238.53125  275.999756
2041-01-13 12:00:00  38.59375  238.53125  277.326630
2041-01-14 12:00:00  38.59375  238.53125  280.185944
2041-01-15 12:00:00  38.59375  238.53125  279.488586
2041-01-16 12:00:00  38.59375  238.53125  278.499573
2041-01-17 12:00:00  38.59375  238.53125  277.

                          lat        lon      tasmax
time                                                
2042-01-01 12:00:00  38.59375  238.53125  291.981201
2042-01-02 12:00:00  38.59375  238.53125  290.925934
2042-01-03 12:00:00  38.59375  238.53125  286.343262
2042-01-04 12:00:00  38.59375  238.53125  288.233795
2042-01-05 12:00:00  38.59375  238.53125  286.897644
2042-01-06 12:00:00  38.59375  238.53125  285.985901
2042-01-07 12:00:00  38.59375  238.53125  285.352753
2042-01-08 12:00:00  38.59375  238.53125  283.863647
2042-01-09 12:00:00  38.59375  238.53125  284.973053
2042-01-10 12:00:00  38.59375  238.53125  284.967224
2042-01-11 12:00:00  38.59375  238.53125  286.263824
2042-01-12 12:00:00  38.59375  238.53125  284.371246
2042-01-13 12:00:00  38.59375  238.53125  288.812531
2042-01-14 12:00:00  38.59375  238.53125  287.507050
2042-01-15 12:00:00  38.59375  238.53125  287.823639
2042-01-16 12:00:00  38.59375  238.53125  287.751740
2042-01-17 12:00:00  38.59375  238.53125  284.